In [1]:
import tensorflow as tf

def setup_gpu():
    physical_devices = tf.config.list_physical_devices('GPU')
    if physical_devices:
        try:
            tf.config.experimental.set_memory_growth(physical_devices[0], True)
            print("config augmentation allocation mémoire gpu activée")
        except RuntimeError as e:
            print(e)

setup_gpu()

config augmentation allocation mémoire gpu activée


In [37]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sea
import sklearn
import scipy as sc
import nltk as nltk
import statsmodels as statsmodels
import os

from sklearn.model_selection import train_test_split

In [36]:
project_path = "C:\\Users\\charl\\Documents\\workspace\\green_ia\\cch\\"

In [3]:
# REMPLACER LE CHEMIN A L AVENIR 
openfoodfacts_csv = "C:\\Users\\charl\\Documents\\workspace\\green_ia\\cch\\data\\openfoodfacts.csv"
df = pd.read_csv(openfoodfacts_csv)

C:\Users\charl\AppData\Local\Temp\ipykernel_7908\3728667497.py:2: DtypeWarning: Columns (0,11,17,23,24,25,26,27,31,32,33,34,35,44,46,50,52,56,67,72) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(openfoodfacts_csv)


In [6]:
# afficher % de nan / colonne:
total_rows = df.shape[0]
nan_counts = df.isna().sum()
nan_percentage = (nan_counts / total_rows) * 100
nan_df = pd.DataFrame({'column_name': nan_percentage.index, 'percentage_nan': nan_percentage.values})
nan_df.head(10)

,column_name,percentage_nan
0,code,0.000000
1,url,0.000000
2,creator,0.000124
3,created_t,0.000000
4,created_datetime,0.000000
5,last_modified_t,0.000000
6,last_modified_datetime,0.000000
7,last_modified_by,3.174734
8,last_updated_t,0.723191
9,last_updated_datetime,0.723191


In [12]:
# affiche le nom de toutes les colonnes du df 
column_list = list(df.columns)
print(f"nbr col: {len(column_list)}, liste: {column_list}")

nbr col: 206, liste: ['code', 'url', 'creator', 'created_t', 'created_datetime', 'last_modified_t', 'last_modified_datetime', 'last_modified_by', 'last_updated_t', 'last_updated_datetime', 'product_name', 'abbreviated_product_name', 'generic_name', 'quantity', 'packaging', 'packaging_tags', 'packaging_en', 'packaging_text', 'brands', 'brands_tags', 'categories', 'categories_tags', 'categories_en', 'origins', 'origins_tags', 'origins_en', 'manufacturing_places', 'manufacturing_places_tags', 'labels', 'labels_tags', 'labels_en', 'emb_codes', 'emb_codes_tags', 'first_packaging_code_geo', 'cities', 'cities_tags', 'purchase_places', 'stores', 'countries', 'countries_tags', 'countries_en', 'ingredients_text', 'ingredients_tags', 'ingredients_analysis_tags', 'allergens', 'allergens_en', 'traces', 'traces_tags', 'traces_en', 'serving_size', 'serving_quantity', 'no_nutrition_data', 'additives_n', 'additives', 'additives_tags', 'additives_en', 'nutriscore_score', 'nutriscore_grade', 'nova_group'

In [13]:
nan_percent_range = [(0, 10), (10, 20), (20, 30), (30, 40), (40, 50), (50, 60), (60, 70), (70, 80), (80, 90), (90, 100)]
grouped = nan_df.groupby(pd.cut(nan_df['percentage_nan'], bins=[tranche[0] for tranche in nan_percent_range + [(100,)]]))

for tranche, group in grouped:
    print(f"range {tranche}:")
    print(group['column_name'].tolist())
    print()

range (0, 10]:
['creator', 'last_modified_by', 'last_updated_t', 'last_updated_datetime', 'product_name', 'countries', 'countries_tags', 'countries_en', 'nutriscore_grade', 'pnns_groups_1', 'pnns_groups_2', 'states', 'states_tags', 'states_en', 'ecoscore_grade', 'completeness']

range (10, 20]:
['last_image_t', 'last_image_datetime', 'image_url', 'image_small_url']

range (20, 30]:
['energy-kcal_100g', 'energy_100g', 'fat_100g', 'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g', 'proteins_100g']

range (30, 40]:
['salt_100g', 'sodium_100g']

range (40, 50]:
['brands', 'brands_tags', 'image_nutrition_url', 'image_nutrition_small_url']

range (50, 60]:
['categories', 'categories_tags', 'categories_en', 'main_category', 'main_category_en']

range (60, 70]:
['quantity', 'ingredients_analysis_tags', 'nutriscore_score', 'food_groups', 'food_groups_tags', 'food_groups_en', 'nutrient_levels_tags', 'product_quantity', 'unique_scans_n', 'popularity_tags', 'fiber_100g', 'nutrition-score-f

C:\Users\charl\AppData\Local\Temp\ipykernel_7908\999090444.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = nan_df.groupby(pd.cut(nan_df['percentage_nan'], bins=[tranche[0] for tranche in nan_percent_range + [(100,)]]))


# Génération df predict catégories

In [40]:
col_classi = [
    'product_name',
    'pnns_groups_1',
    'pnns_groups_2',
    'ecoscore_grade',
    'energy-kcal_100g',
    'energy_100g',
    'fat_100g',
    'saturated-fat_100g',
    'carbohydrates_100g',
    'sugars_100g',
    'proteins_100g',
    'salt_100g',
    'sodium_100g',
    'brands_tags',
    'categories_tags',
    'main_category_en'
]
# récupération des colonnes intéressantes
cat_df = df[col_classi] 

# suppresion des lignes où pnns2 = unknow ou NaN
cat_df = cat_df.dropna(subset = ["pnns_groups_2"])
cat_df = cat_df[cat_df['pnns_groups_2'] != 'unknown']

# mélange des lignes aléatoirement 
cat_df = cat_df.sample(frac=1).reset_index(drop=True)

# split des df de train, test et valid
cat_train_df, cat_test_df = train_test_split(cat_df, test_size=0.1, random_state=42)
cat_train_df, cat_valid_df = train_test_split(cat_train_df, test_size=0.05, random_state=42) 

# sauvegarde au format csv 
cat_train_df.to_csv(project_path + '\\data\\cat_train_df_01.csv', index=False)
cat_test_df.to_csv(project_path + '\\data\\cat_test_df_01.csv', index=False)
cat_valid_df.to_csv(project_path + '\\data\\cat_valid_df_01.csv', index=False)

cat_train_df.tail(5)

,product_name,pnns_groups_1,pnns_groups_2,ecoscore_grade,energy-kcal_100g,energy_100g,fat_100g,saturated-fat_100g,carbohydrates_100g,sugars_100g,proteins_100g,salt_100g,sodium_100g,brands_tags,categories_tags,main_category_en
1134043,Temptation creamy,Sugary snacks,Sweets,unknown,529.0,2213.0,39.00,13.00,30.00,2.00,23.00,0.2300,0.09200,4-nutrition,"en:breakfasts,en:spreads,en:specific-products,...",Products for specific diets
469974,Dipping Oil,Fat and sauces,Dressings and sauces,unknown,800.0,3347.0,93.33,13.33,0.00,0.00,0.00,0.7500,0.30000,renaissance-food-group-llc,"en:condiments,en:sauces,en:dips,en:groceries",Groceries
702817,Organic penne pasta macaroni product,Cereals and potatoes,Cereals,unknown,375.0,1569.0,1.79,0.00,73.21,3.57,12.50,0.0000,0.00000,NaN,"en:plant-based-foods-and-beverages,en:plant-ba...",Pastas
355776,"100% tart cherry juice, 100% tart",Beverages,Unsweetened beverages,unknown,42.0,176.0,0.00,0.00,10.42,10.42,0.42,0.0000,0.00000,NaN,"en:plant-based-foods-and-beverages,en:beverage...",Plant-based beverages
770613,"Der Sahnige mit Korinthen, getrockneten Feigen...",Milk and dairy products,Milk and yogurt,b,136.0,935.0,9.43,5.60,9.07,8.64,3.71,0.0929,0.03716,edeka,"en:dairies,en:fermented-foods,en:fermented-mil...",Greek-style yogurts
